In [ ]:
from icecube import icetray, dataclasses, dataio, recclasses
import numpy as np
import glob
import json
import I3Tray
from realtime_hese.python.HESE_alerts_v2 import *

files = glob.glob("./search_data/201*/Alerts*.i3")
files.sort()
print(files)

outfile = 'not_all_hese_gfu.i3'


In [ ]:
def find_v2_alerts(frame):
    keeper = False
    if frame.Has("Alert_JSON"):
        gfu_alert = json.loads(frame['Alert_JSON'].value)
        if gfu_alert['pass_tight']:
            keeper = True
        else:
            if gfu_alert['pass_loose']:
                keeper=True
    fm_hese = frame['FilterMask']['HESEFilter_15']
    if fm_hese.condition_passed:
        cqtot = frame['HESE_CausalQTot'].value
        ldir = round(frame['OnlineL2_SplineMPE_DirectHitsB'].dir_track_length,5)
        zen = round(frame["OnlineL2_SplineMPE"].dir.zenith, 5)
        cos_zen = np.cos(zen)
        llh_ratio =round(frame["HESE_llhratio"].value, 5)
        pass_precut = (cqtot > 6000.) and (ldir > 200.)
        pass_loose  = pass_precut and np.asscalar(HESE_track_selection_soft(cqtot, llh_ratio, cos_zen))
        pass_tight  = pass_precut and np.asscalar(HESE_track_selection_hard(cqtot, llh_ratio, cos_zen))
        if pass_loose:
            keeper=True
        if pass_tight:
            keeper=True
    return not keeper

In [ ]:
tray = I3Tray.I3Tray()
tray.AddModule("I3Reader", "reader", filenameList=files)
tray.AddModule(find_v2_alerts,'find_alerts',Streams=[icetray.I3Frame.Physics])
tray.AddModule("I3Writer","wrat",DropOrphanStreams=[icetray.I3Frame.DAQ],
               Streams=[icetray.I3Frame.DAQ, icetray.I3Frame.Physics],
               filename = outfile)
tray.Execute()